In [1]:
pip install opencv-python face-recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=ec86879ac50ffcc184fd9801b923b2495ebf33ec3b18893316f53d77e1984bc7
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [ ]:
import os
import cv2
import face_recognition
import pandas as pd

In [ ]:
student = pd.read_csv('student.csv')
known_encodings = []
known_names = []

In [ ]:
for img_file in os.listdir('student'):
    img = cv2.imread(os.path.join('student', img_file))
    encoding = face_recognition.face_encodings(img)[0]
    known_encodings.append(encoding)
    known_names.append(os.path.splitext(img_file)[0])

In [ ]:
from fer import FER
emotion_detector = FER()

In [ ]:
attendance = {
    name:{" Present":False, "Emotion":"Unknown","Time":None} for name in known_names
}


In [11]:
import datetime


In [13]:
def run_attendance_system():
  current_time = datetime.datetime.now().time()
  start_time = datetime.time(9,30)
  end_time = datetime.time(10,0)

  if not (start_time<=current_time <= end_time):
    print("Outside the hours")
    return

  cap = cv2.VideoCapture(0)

  while datetime.datetime.now().time() < end_time:
    ret, frame = cap.read()
    if not ret:
      break

    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
      matches = face_recognition.compare_faces(known_encodings, face_encoding)
      name = "Unknown"
      if True in matches:
        first_match_index = matches.index(True)
        name = known_names[first_match_index]

        roi = frame[top:bottom, left:right]
        emotion, score = emotion_detector.top_emotion(roi)

        attendance[name]["Present"] = True
        attendance[name]["Emotion"] = emotion
        attendance[name]["Time"] = datetime.datetime.now().strftime("%H:%M:%S")

      cv2.rectangle(frame,(left,top),(right,bottom),(0,255,0),2)
      cv2.putText(frame,f"{name}:{emotion}",(left,top-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),2)

    cv2.imshow("Attendance System",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()

  output = pd.DataFrame(attendance).T.reset_index().rename(columns={"index":"Name"})
  output["Present"] = output["Present"].map({True: "Present", False: "Absent"})
  output.to_csv("attendance_report.csv", index = False)